In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType

In [0]:
trips = spark.table("default.staging_trips")
display(trips.limit(10))

trip_id,rideable_type,start_at,ended_at,start_station_id,end_station_id,rider_id
89E7AA6C29227EFF,classic_bike,2021-02-12T16:14:56.000+0000,2021-02-12T16:21:43.000+0000,525,660,71934
0FEFDE2603568365,classic_bike,2021-02-14T17:52:38.000+0000,2021-02-14T18:12:09.000+0000,525,16806,47854
E6159D746B2DBB91,electric_bike,2021-02-09T19:10:18.000+0000,2021-02-09T19:19:10.000+0000,KA1503000012,TA1305000029,70870
B32D3199F1C2E75B,classic_bike,2021-02-02T17:49:41.000+0000,2021-02-02T17:54:06.000+0000,637,TA1305000034,58974
83E463F23575F4BF,electric_bike,2021-02-23T15:07:23.000+0000,2021-02-23T15:22:37.000+0000,13216,TA1309000055,39608
BDAA7E3494E8D545,electric_bike,2021-02-24T15:43:33.000+0000,2021-02-24T15:49:05.000+0000,18003,KP1705001026,36267
A772742351171257,classic_bike,2021-02-01T17:47:42.000+0000,2021-02-01T17:48:33.000+0000,KP1705001026,KP1705001026,50104
295476889D9B79F8,classic_bike,2021-02-11T18:33:53.000+0000,2021-02-11T18:35:09.000+0000,18003,18003,19618
362087194BA4CC9A,classic_bike,2021-02-27T15:13:39.000+0000,2021-02-27T15:36:36.000+0000,KP1705001026,KP1705001026,16732
21630F715038CCB0,classic_bike,2021-02-20T08:59:42.000+0000,2021-02-20T09:17:04.000+0000,KP1705001026,KP1705001026,57068


In [0]:
# Check whether trip_id is duplicated
print(trips.count())
fact_trips = trips.dropDuplicates(["trip_id"])
print(fact_trips.count())

4584921
4584921

In [0]:
riders = spark.table("default.staging_riders")
display(riders.limit(10))

rider_id,first_name,last_name,address,birthday,account_start_date,account_end_date,is_member
1000,Diana,Clark,1200 Alyssa Squares,1989-02-13T00:00:00.000+0000,2019-04-23T00:00:00.000+0000,null,true
1001,Jennifer,Smith,397 Diana Ferry,1976-08-10T00:00:00.000+0000,2019-11-01T00:00:00.000+0000,2020-09-01T00:00:00.000+0000,true
1002,Karen,Smith,644 Brittany Row Apt. 097,1998-08-10T00:00:00.000+0000,2022-02-04T00:00:00.000+0000,null,true
1003,Bryan,Roberts,996 Dickerson Turnpike,1999-03-29T00:00:00.000+0000,2019-08-26T00:00:00.000+0000,null,false
1004,Jesse,Middleton,7009 Nathan Expressway,1969-04-11T00:00:00.000+0000,2019-09-14T00:00:00.000+0000,null,true
1005,Christine,Rodriguez,224 Washington Mills Apt. 467,1974-08-27T00:00:00.000+0000,2020-03-24T00:00:00.000+0000,null,false
1006,Alicia,Taylor,1137 Angela Locks,2004-01-30T00:00:00.000+0000,2020-11-27T00:00:00.000+0000,2021-12-01T00:00:00.000+0000,true
1007,Benjamin,Fernandez,979 Phillips Ways,1988-01-11T00:00:00.000+0000,2016-12-11T00:00:00.000+0000,null,false
1008,John,Crawford,7691 Evans Court,1987-02-21T00:00:00.000+0000,2021-03-28T00:00:00.000+0000,2021-07-01T00:00:00.000+0000,true
1009,Victoria,Ritter,9922 Jim Crest Apt. 319,1981-02-07T00:00:00.000+0000,2020-06-12T00:00:00.000+0000,2021-11-01T00:00:00.000+0000,true


In [0]:
fact_trips = fact_trips.alias("tbl1").join(riders.alias("tbl2"), col("tbl1.rider_id") == col("tbl2.rider_id"), "left") \
            .withColumn('duration_seconds', round((unix_timestamp("ended_at") - unix_timestamp('start_at')))) \
            .withColumn('rider_age', (months_between(col("account_start_date"), col("birthday")) / 12).cast('int')) \
            .select("trip_id", col("tbl1.rider_id"), "rideable_type", "start_station_id", "end_station_id", "start_at", "ended_at", "duration_seconds", "rider_age")
            
display(fact_trips.limit(10))

trip_id,rider_id,rideable_type,start_station_id,end_station_id,start_at,ended_at,duration_seconds,rider_age
000002EBE159AE82,31773,electric_bike,638,13033,2021-06-22T17:25:15.000+0000,2021-06-22T17:31:34.000+0000,379,43
0000080D43BAA9E4,21335,classic_bike,624,SL-008,2021-08-29T15:38:05.000+0000,2021-08-29T16:24:03.000+0000,2758,22
00000CAE95438C9D,71748,classic_bike,13022,TA1305000003,2021-07-20T15:40:46.000+0000,2021-07-20T17:38:17.000+0000,7051,22
00000EBBC119168C,44951,classic_bike,KA1503000069,TA1309000037,2021-10-31T11:30:37.000+0000,2021-10-31T11:39:27.000+0000,530,24
000019B7F053D461,66662,classic_bike,13193,TA1309000033,2021-08-13T19:57:28.000+0000,2021-08-13T20:02:56.000+0000,328,14
00001A81D056B01B,14172,classic_bike,13432,TA1306000015,2021-04-14T08:10:11.000+0000,2021-04-14T08:19:14.000+0000,543,24
00001B4F79D102B5,43153,classic_bike,TA1309000049,13325,2021-07-28T07:58:27.000+0000,2021-07-28T08:05:00.000+0000,393,29
00001BEE76AB24E0,19382,electric_bike,TA1306000010,13061,2021-11-30T16:55:38.000+0000,2021-11-30T17:08:53.000+0000,795,22
00001DCF2BC423F4,21478,docked_bike,13008,TA1307000048,2021-06-13T12:00:49.000+0000,2021-06-13T12:29:51.000+0000,1742,37
000020C92AA9D6F7,63388,classic_bike,13128,KA1503000064,2021-09-12T09:53:00.000+0000,2021-09-12T10:12:52.000+0000,1192,20


In [0]:
fact_trips.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("default.fact_trips")